In [1]:
# traing data into this folder
!ls /veld/input/

clean_de_wiki.csv  clean_petite.csv  dewiki.txt  petite.csv


In [2]:
# code here
!ls /veld/executable/

train.ipynb


In [3]:
# save models here
!ls /veld/output/

model


New approach 20240426

In [10]:
import pandas as pd
import re

In [1]:
petite= pd.read_csv("/veld/input/de_wiki.csv")
petite = petite.rename(columns={"\ttext": "text"})

petite_cleaned = petite.copy()

In [2]:
for i in range(len(petite)):
    text = petite.iloc[i]["text"]
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\t", "", text)
    text = re.sub(r"“", "", text)
    text = re.sub(r"„", "", text)
    petite_cleaned.iloc[i]["text"] = text #we load the data from the left into the variety to the right

In [3]:
petite.head()

,text
0,0\tAlan Smithee steht als Pseudonym für einen ...
1,1\tActinium ist ein radioaktives chemisches El...
2,2\tAng Lee Oktober in Chaozhou Landkreis Pingt...
3,3\tAnschluss ist in der Soziologie ein Fachbeg...
4,4\tDie Aussagenlogik ist ein Teilgebiet der Lo...


In [4]:
petite_cleaned.head()

,text
0,Alan Smithee steht als Pseudonym für einen fik...
1,Actinium ist ein radioaktives chemisches Eleme...
2,Ang Lee Oktober in Chaozhou Landkreis Pingtung...
3,Anschluss ist in der Soziologie ein Fachbegrif...
4,Die Aussagenlogik ist ein Teilgebiet der Logik...


In [5]:
petite_cleaned.sample(5)

,text
435473,Das Kastell Würzberg in der älteren Literatur ...
291377,Robert Kurt Kudielka August in Lindau Bodensee...
152320,Uebigau Wahrenbrück ist eine Stadt im Landkrei...
448715,Ken Leung Januar in New York City ist ein US a...
416056,Juliette ist ein französischer weiblicher Vorn...


In [3]:
TRAIN_DATA_PATH = "/veld/output/clean/de_wiki_cleaned.csv"
MODEL_PATH = "/veld/output/model/20240426word2vec_de.model"
MODEL_METADATA_PATH = "/veld/output/model/metadata.yaml"
TRAIN_DATA_NAME = "german wikipedia, csv"
VECTOR_SIZE = 200
WINDOW = 3
MIN_COUNT = 1
WORKERS = 10

In [11]:
petite_cleaned.to_csv(TRAIN_DATA_PATH)

In [29]:
import gensim
from gensim.utils import simple_preprocess
from datetime import datetime

In [35]:
class Iterable:

    def __init__(self, csv_file_path):
        self.df = pd.read_csv(csv_file_path)
    
    def __iter__(self):
        for line in self.df.iterrows():
            yield simple_preprocess(line[1]["text"])

In [36]:
iter = Iterable(TRAIN_DATA_PATH)

In [37]:
time_start = datetime.now()
model = gensim.models.Word2Vec (iter,
                                vector_size=VECTOR_SIZE,
                                window=WINDOW,
                                min_count=MIN_COUNT,
                                workers=WORKERS)
time_end = datetime.now()
duration = (time_end - time_start).seconds / 60
model.save(MODEL_PATH)

KeyboardInterrupt: 

In [ ]:
import os
# calculate size of training data
def calc_size(file):
    size = os.path.getsize(file)
    for unit in ["","KB","MB","GB","TB"]:
        if abs(size) < 1024.0:
            return f"{round(size, 1)} {unit}"
        size /= 1024.0
train_data_size = calc_size(TRAIN_DATA_PATH)
model_data_size = calc_size(MODEL_PATH)

# calculate hash of training data
train_data_md5_hash = subprocess.run(["md5sum", TRAIN_DATA_PATH], capture_output=True, text=True)
train_data_md5_hash = train_data_md5_hash.stdout.split()[0]


# aggregate into metadata dictionary
metadata = {
    "train_data_name": TRAIN_DATA_NAME,
    "train_data_size": train_data_size,
    "train_data_md5_hash": train_data_md5_hash,
    "training_vector_size": VECTOR_SIZE,
    "window": WINDOW,
    "min_count": MIN_COUNT,
    "workers": WORKERS,
    "training_duration (minutes)": round(duration, 1),
    "model_data_size": model_data_size,
}

# write to yaml
with open(MODEL_METADATA_PATH, "w") as f:
    # iteration over dictionary to ensure the yaml writer respects the order
    for k, v in metadata.items():
        yaml.dump({k: v}, f)

In [ ]:
word ="Wien"
print(f'Similaries to {word}:')
for similar_word, similarity in model.wv.most_similar(word, topn=10):
    print(f"- {similar_word}: {similarity}")

In [ ]:
word ="Frau"
print(f'Similaries to {word}:')
for similar_word, similarity in model.wv.most_similar(word, topn=10):
    print(f"- {similar_word}: {similarity}")

In [ ]:
word ="Dame"
print(f'Similaries to {word}:')
for similar_word, similarity in model.wv.most_similar(word, topn=10):
    print(f"- {similar_word}: {similarity}")

In [ ]:
word ="Baum"
print(f'Similaries to {word}:')
for similar_word, similarity in model.wv.most_similar(word, topn=10):
    print(f"- {similar_word}: {similarity}")

In [ ]:
model.wv.similarity('Frau', 'Dame')

In [ ]:
model.wv['Frau']

In [ ]:
import numpy

In [ ]:
print(cosine_similarity)

In [ ]:
model.wv.similarity('Frau', 'Baum')

In [ ]:
model.wv.similarity('Dame', 'Baum')

In [ ]:
# Function to calculate cosine similarity between two word vectors
def cosine_similarity(word1, word2):
    try:
        # Calculate cosine similarity between the word vectors
        similarity_score = model.wv.similarity(word1, word2)
        return similarity_score
    except KeyError:
        # Handle the case where one or both words are not in the vocabulary
        return None

# Example usage
word1 = 'Frau'
word2 = 'Dame'

# Calculate cosine similarity between 'Frau' and 'Dame'
similarity_score = cosine_similarity(word1, word2)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word1}' and '{word2}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word1}' or '{word2}' is not in the vocabulary.")


In [ ]:
# Example usage
word3 = 'Ball'
word4 = 'Fest'

# Calculate cosine similarity between 'Ball' and 'Fest'
similarity_score = cosine_similarity(word3, word4)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word3}' and '{word4}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word3}' or '{word4}' is not in the vocabulary.")

In [ ]:
# Example usage
word5 = 'Gala'
word6 = 'Fest'

# Calculate cosine similarity between 'Gala' and 'Fest'
similarity_score = cosine_similarity(word5, word6)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word5}' and '{word6}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word5}' or '{word6}' is not in the vocabulary.")

In [ ]:
# Example usage
word7 = 'Tanz'
word8 = 'Fest'

# Calculate cosine similarity between 'Tanz' and 'Fest'
similarity_score = cosine_similarity(word7, word8)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word7}' and '{word8}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word7}' or '{word8}' is not in the vocabulary.")

In [ ]:
# Example usage
word9 = 'Gala'
word10 = 'Tanz'

# Calculate cosine similarity between 'Gala' and 'Fest'
similarity_score = cosine_similarity(word9, word10)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word9}' and '{word10}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word9}' or '{word10}' is not in the vocabulary.")

In [ ]:
# Example usage
word11 = 'Ball'
word12 = 'Tanz'

# Calculate cosine similarity between 'Gala' and 'Fest'
similarity_score = cosine_similarity(word11, word12)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word11}' and '{word12}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word11}' or '{word12}' is not in the vocabulary.")

In [ ]:
# Example usage
word13 = 'Ball'
word14 = 'Gala'

# Calculate cosine similarity between 'Gala' and 'Fest'
similarity_score = cosine_similarity(word13, word14)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word13}' and '{word14}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word13}' or '{word14}' is not in the vocabulary.")

In [ ]:
# Example usage
word15 = 'Zweck'
word16 = 'Ziel'

# Calculate cosine similarity between 'Gala' and 'Fest'
similarity_score = cosine_similarity(word15, word16)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word15}' and '{word16}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word15}' or '{word16}' is not in the vocabulary.")

In [ ]:
# Example usage
word17 = 'Feuer'
word18 = 'Brand'

# Calculate cosine similarity between 'Gala' and 'Fest'
similarity_score = cosine_similarity(word17, word18)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word17}' and '{word18}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word17}' or '{word18}' is not in the vocabulary.")

In [ ]:
# Example usage
word19 = 'Haltestelle'
word20 = 'Station'

# Calculate cosine similarity between 'Gala' and 'Fest'
similarity_score = cosine_similarity(word19, word20)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word19}' and '{word20}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word19}' or '{word20}' is not in the vocabulary.")

In [ ]:
# Example usage
word21 = 'Zahl'
word22 = 'Nummer'

# Calculate cosine similarity between 'Gala' and 'Fest'
similarity_score = cosine_similarity(word21, word22)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word21}' and '{word22}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word21}' or '{word22}' is not in the vocabulary.")

In [ ]:
# Example usage
word25 = 'Raum'
word26 = 'Zimmer'

# Calculate cosine similarity between 'Gala' and 'Fest'
similarity_score = cosine_similarity(word25, word26)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word25}' and '{word26}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word25}' or '{word26}' is not in the vocabulary.")

In [ ]:
# Example usage
word27 = 'Etage'
word28 = 'Stock'

# Calculate cosine similarity between 'Gala' and 'Fest'
similarity_score = cosine_similarity(word27, word28)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word27}' and '{word28}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word27}' or '{word28}' is not in the vocabulary.")

In [ ]:
# Example usage
word27 = 'Ball'
word28 = 'Kugel'

# Calculate cosine similarity between 'Gala' and 'Fest'
similarity_score = cosine_similarity(word27, word28)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word27}' and '{word28}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word27}' or '{word28}' is not in the vocabulary.")

In [ ]:
# Example usage
word27 = 'Frau'
word28 = 'Mann'

# Calculate cosine similarity between 'Gala' and 'Fest'
similarity_score = cosine_similarity(word27, word28)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word27}' and '{word28}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word27}' or '{word28}' is not in the vocabulary.")

In [ ]:
# Example usage
word27 = 'Frau'
word28 = 'Dame'

# Calculate cosine similarity between 'Gala' and 'Fest'
similarity_score = cosine_similarity(word27, word28)

if similarity_score is not None:
    print(f"Cosine Similarity between '{word27}' and '{word28}': {similarity_score:.4f}")
else:
    print(f"At least one of the words '{word27}' or '{word28}' is not in the vocabulary.")

In [ ]:
import gensim

In [ ]:
model = gensim.models.Word2Vec.load("/veld/output/model/20240425word2vec_de.model")

In [ ]:
model.wv.similarity('Frau', 'Dame')